In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -qqq transformers
!pip install -qqq sentencepiece

In [ ]:
import os
import gc
import cv2
import copy
import time
import random
import joblib

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import DataCollatorWithPadding

# Utils
from tqdm import tqdm

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
CONFIG = dict(
    seed = 42,
    model_name = "microsoft/deberta-v3-large",
    test_batch_size = 16,
    max_length = 512,
    num_classes = 4,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    exp = "057(MultiLabel)",
    n_fold = 4,
    BEST_F1 = False,
)

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
l = ['</em>',
 '<span>',
 '<ul>',
 '</p>',
 '</ul>',
 '<li>',
 '<p>',
 '</li>',
 '<em>',
 '</ol>',
 '</span>']
 
CONFIG["tokenizer"].add_tokens(l, special_tokens=True)

11

In [ ]:
MODEL_PATHS = []

for fold in range(CONFIG["n_fold"]):
    if CONFIG["BEST_F1"]:
        MODEL_PATHS.append(f'/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/model/exp_{CONFIG["exp"]}/BestF1Model/Loss-Fold-{fold}.bin')
    else:
        MODEL_PATHS.append(f'/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/model/exp_{CONFIG["exp"]}/Loss-Fold-{fold}.bin')

MODEL_PATHS

['/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/model/exp_057(MultiLabel)/Loss-Fold-0.bin',
 '/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/model/exp_057(MultiLabel)/Loss-Fold-1.bin',
 '/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/model/exp_057(MultiLabel)/Loss-Fold-2.bin',
 '/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/model/exp_057(MultiLabel)/Loss-Fold-3.bin']

In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/input/train.csv"
TEST_DIR = "/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/input/test.csv"

In [ ]:
class SignateDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.description = df["description"].values
        # self.targets = df['jobflag'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        description = self.description[index]
        inputs = self.tokenizer.encode_plus(
                        description,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len
                    )
        
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            # 'target': self.targets[index]
        }

In [ ]:
df = pd.read_csv(TEST_DIR)
sample = pd.read_csv("/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/input/submit_sample.csv", header=None)
sample.head()

,0,1
0,1516,1
1,1517,1
2,1518,1
3,1519,1
4,1520,1


In [ ]:
collate_fn = DataCollatorWithPadding(tokenizer=CONFIG['tokenizer'])

In [ ]:
test_dataset = SignateDataset(df, CONFIG['tokenizer'], max_length=CONFIG['max_length'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                         num_workers=2, shuffle=False, pin_memory=True, collate_fn=collate_fn)

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [ ]:
class SignateModel(nn.Module):
    def __init__(self, model_name):
        super(SignateModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.pooler = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.pooler(out.last_hidden_state, mask)
        out = self.drop(out)
        outputs = self.fc(out)
        return outputs

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        outputs = F.softmax(outputs, dim=1)
        PREDS.append(outputs.cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = SignateModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
preds = inference(MODEL_PATHS, test_loader, CONFIG['device'])
sample.iloc[:, 1] = np.argmax(preds, axis=1) + 1

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 95/95 [01:17<00:00,  1.23it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

Getting predictions for model 2


100%|██████████| 95/95 [01:09<00:00,  1.36it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

Getting predictions for model 3


100%|██████████| 95/95 [01:09<00:00,  1.36it/s]
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

Getting predictions for model 4


100%|██████████| 95/95 [01:09<00:00,  1.36it/s]


In [ ]:
train = pd.read_csv(TRAIN_DIR)
test = pd.read_csv(TEST_DIR)
dup_train = train[train["description"].isin(test["description"])][["description", "jobflag"]]
dup_test = test[test["description"].isin(train["description"])]
dup_df = pd.merge(dup_test, dup_train, on="description", how="left")[["id", "jobflag"]]

for id, flag in zip(dup_df["id"], dup_df["jobflag"]):
    sample.loc[(sample.iloc[:, 0] == id), 1] = flag


sample

,0,1
0,1516,1
1,1517,4
2,1518,3
3,1519,4
4,1520,3
...,...,...
1512,3028,3
1513,3029,1
1514,3030,3
1515,3031,4


In [ ]:
if CONFIG["BEST_F1"]:
    sample.to_csv(f"/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/output/exp_{CONFIG['exp']}_BEST_F1.csv", index=False)
    sample.to_csv(f'exp_{CONFIG["exp"]}_BEST_F1.csv', index=False)
else:
    sample.to_csv(f"/content/drive/MyDrive/Competitions/SIGNATE/STUDENT_CUP/penguin/output/exp_{CONFIG['exp']}.csv", index=False)
    sample.to_csv(f'exp_{CONFIG["exp"]}.csv', index=False)